### DeOldify Config

#### Import libs

In [ ]:
# Import DeOldfy libraries
import sys
sys.path.append('../')
from deoldify import device
from deoldify.device_id import DeviceId
from deoldify.visualize import *
import warnings

from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image

%matplotlib inline

# Set Deoldfy configs
device.set(device=DeviceId.CPU)
torch.backends.cudnn.benchmark=True
plt.style.use('dark_background')
plt.style
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

#### Create Model

In [ ]:
# Create DeOldfy Model
colorizer = get_image_colorizer(artistic=True)

#### Load Input Image

In [ ]:
input_img = Image.open('../test_images/image.png')


#### Define preprocess function

In [ ]:
import torchvision.transforms as tf

transform_in = tf.Compose([
    tf.Resize((560,560)),
    tf.ToTensor(),
    tf.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transformed_img = transform_in(input_img).unsqueeze(0)

In [ ]:
# Run model
with torch.no_grad():
    output = colorizer.filter.filters[0].learn.model(transformed_img)[0]
print(output.shape, output.data[0,0,0])

In [ ]:
transform_out = tf.Compose([
    tf.Normalize([0., 0., 0.], [1/0.229, 1/0.224, 1/0.225]),
    tf.Normalize([-0.485, -0.456, -0.406], [ 1., 1., 1. ]),
    tf.Resize((input_img.size[1],input_img.size[0])),
    # lambda x: x*255,
    tf.ToPILImage(),
])
transformed_output = transform_out(output)
# output_img = transformed_output.permute(1,2,0).numpy().astype(np.uint8)
# output_img = Image.fromarray(output_img)
# output_img = output_img.resize(input_img.size, resample=Image.BILINEAR)

In [ ]:
import cv2
color_np = np.array(transformed_output)
orig_np = np.array(input_img)
color_yuv = cv2.cvtColor(color_np, cv2.COLOR_RGB2YUV)
# do a black and white transform first to get better luminance values
orig_yuv = cv2.cvtColor(orig_np, cv2.COLOR_RGB2YUV)
hires = np.copy(orig_yuv)
hires[:, :, 1:3] = color_yuv[:, :, 1:3]
final = cv2.cvtColor(hires, cv2.COLOR_YUV2RGB)
final = Image.fromarray(final)
imshow(final)

In [ ]:
# Convert to TorchScript
trace = torch.jit.trace(colorizer.filter.filters[0].learn.model, transformed_img)

In [ ]:
# Convert to CoreML
import coremltools as ct
mlmodel = ct.convert(
	trace,
	inputs=[ct.TensorType(name="input", shape=transformed_img.shape)],
    convert_to='neuralnetwork'
)

In [ ]:
# Make a prediction using Core ML
out_mlmodel = mlmodel.predict({"input": transformed_img})

In [ ]:
output = out_mlmodel['var_1244'][0]

output_img = transform_out(torch.tensor(output))

In [ ]:
import cv2
import matplotlib.pyplot as plt

color_np = np.array(output_img)
orig_np = np.array(input_img)
color_yuv = cv2.cvtColor(color_np, cv2.COLOR_RGB2YUV)
# do a black and white transform first to get better luminance values
orig_yuv = cv2.cvtColor(orig_np, cv2.COLOR_RGB2YUV)
hires = np.copy(orig_yuv)
hires[:, :, 1:3] = color_yuv[:, :, 1:3]
final = cv2.cvtColor(hires, cv2.COLOR_YUV2RGB)
final = Image.fromarray(final)
f, axarr = plt.subplots(1,2, figsize=(40,40))
axarr[0].imshow(input_img)
axarr[0].axis('off')
axarr[1].imshow(final)
axarr[1].axis('off')